In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TerrenceZwy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
import re
import nltk
import os
import numpy as np
import codecs
import multiprocessing as mp
import pandas as pd

from nltk.corpus import stopwords
from collections import Counter
from typing import Dict, List, Tuple

from nltk import stem

stemmer = stem.PorterStemmer()

# find the path of emails
def listdir(directory: str) -> List:
    """
    A specialized version of os.listdir() that ignores files that
    start with a leading period.
    
    Especially dismissing .DS_STORE s.
    """
    filelist = os.listdir(directory)
    return [x for x in filelist if not (x.startswith('.'))]

stopwords = stopwords.words('english')
cut_model = nltk.WordPunctTokenizer()


# Preprocess emails
def enron_processor(emails_dir: str, return_list: list) -> list:    
    """
    * remove numbers
    * remove stopwords
    * add lables
    """
    dirs = [os.path.join(emails_dir, f) for f in os.listdir(emails_dir)]
    for d in dirs:
        emails = [os.path.join(d, f) for f in os.listdir(d)]
        for mail in emails:
            # print(mail)
            with codecs.open(mail, "rb", encoding='utf_8_sig', errors='ignore') as m:
                email_list = []
                line_str = ""
                for line in m:
                    for word in line:
                        if word.startswith("http"):
                            print(word)
                            word = "URL"
                            print(word)
                        word = stemmer.stem(word)
                    line = re.sub(r'[^a-zA-Z\s]', '', string=line)
                    line = line.lower()
                    line = line.strip()
                    tokens = cut_model.tokenize(line)
                    line = [stemmer.stem(token) for token in tokens if token not in stopwords]
           
                    line = ' '.join(line)
                    line_str = line_str+line+" "
                email_list.append(line_str)
                
                if mail.split(".")[-2] == 'spam':
                    email_list.append("spam")
                else: 
                    email_list.append("ham")
                email_list.append(mail)
                return_list.append(email_list)
                

                
return_dict = {}   
root_dir = 'spampy/datasets/enron'
emails_dirs = [os.path.join(root_dir, f) for f in listdir(root_dir)]
return_list = []
for emails_dir in emails_dirs:
    enron_processor(emails_dir, return_list)


messages = pd.DataFrame(return_list, columns=['message', 'label', 'path'])


In [11]:
messages.to_csv("messages.csv")

In [1]:
import csv
import re
import nltk
import os
import numpy as np
import codecs
import multiprocessing as mp
import pandas as pd



csv.field_size_limit(500 * 1024 * 1024)
with open("messages.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    messages = []
    for row in reader:
        messages.append(row[1:])
messages = messages[1:]
messages = pd.DataFrame(messages, columns=['message', 'label', 'path'])
messages

,message,label,path
0,christma tree farm pictur,ham,spampy/datasets/enron\enron1\ham\0001.1999-12-...
1,vastar resourc inc gari product high island la...,ham,spampy/datasets/enron\enron1\ham\0002.1999-12-...
2,calpin daili ga nomin calpin daili ga nomin doc,ham,spampy/datasets/enron\enron1\ham\0003.1999-12-...
3,issu fyi see note alreadi done stella forward ...,ham,spampy/datasets/enron\enron1\ham\0004.1999-12-...
4,meter nov alloc fyi forward lauri allen hou ec...,ham,spampy/datasets/enron\enron1\ham\0005.1999-12-...
...,...,...,...
33711,iso q good news c edaliss val edumm vl eoggra ...,spam,spampy/datasets/enron\enron6\spam\5995.2005-07...
33712,prescript medicin special precis put buck back...,spam,spampy/datasets/enron\enron6\spam\5997.2005-07...
33713,next gener onlin pharmaci readi rock let man r...,spam,spampy/datasets/enron\enron6\spam\5998.2005-07...
33714,bloow time time learn last time longer bed rea...,spam,spampy/datasets/enron\enron6\spam\5999.2005-07...


In [2]:
# Split train and test (4:1) and store the path in x_train_path

from sklearn.model_selection import train_test_split
messages['label'] = messages['label'].replace('ham', 0)
messages['label'] = messages['label'].replace('spam', 1)

messages_label = messages['label']
message_path = messages['path']
x = messages['message']
y = messages_label

x_train_path = []
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#print(type(x_train))
for i, v in x_train.items():
    x_train_path.append((i, message_path[i]))

x_test_path = []
for i, v in x_test.items():
    x_test_path.append((i, message_path[i]))

In [3]:
x_train

16866     bill desk desk deal number check w volum mgmt...
8409     durasoft java class think option sg forward st...
21190    girl want meet tonight welcom planet largest n...
11095    fw organiz chang east power origin messag enro...
5098     nebenarbeit al gehilf ref sehr geehrter zuk nf...
                               ...                        
33218    read messag pleasur alway dreamt rock hard ere...
22411    rfp diagnosi prognosi algorithm research pleas...
334      hpl system train daren would like session sepa...
4743     need track k hi reciev email last week happi t...
30913    vote jerald want know porn star stay hard cum ...
Name: message, Length: 26972, dtype: object

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tf-idf for train datasets
vect = TfidfVectorizer()
tfidf_train = vect.fit_transform(x_train)
tfidf_matrix_train = pd.DataFrame(tfidf_train.toarray(), columns = vect.get_feature_names())
headers = vect.get_feature_names()
tfidf_matrix_train

,aa,aaa,aaaa,aaaaci,aaaahhhhhh,aaadrizzl,aaaenerfax,aaal,aaasash,aab,...,zzucpkow,zzw,zzx,zzxtfeerekvwkug,zzxxst,zzyudgpd,zzzglvaa,zzzxlqbha,zzzz,zzzzcard
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Tf-idf for test datsets
tfidf_test = vect.transform(x_test)
tfidf_matrix_test = pd.DataFrame(tfidf_test.toarray(), columns = vect.get_feature_names())
tfidf_matrix_test

,aa,aaa,aaaa,aaaaci,aaaahhhhhh,aaadrizzl,aaaenerfax,aaal,aaasash,aab,...,zzucpkow,zzw,zzx,zzxtfeerekvwkug,zzxxst,zzyudgpd,zzzglvaa,zzzxlqbha,zzzz,zzzzcard
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from sklearn.model_selection import KFold
from secml.data import CDataset
from secml.data.splitter import CDataSplitterKFold
from secml.ml.classifiers import CClassifierSVM
from secml.ml.peval.metrics import CMetricAccuracy
from secml.ml.peval.metrics import CMetricConfusionMatrix

from secml.ml.classifiers.multiclass import CClassifierMulticlassOVA
from secml.ml.kernels import CKernelLinear

nb_col = tfidf_train.size

tr_set = CDataset(tfidf_train, y_train)


# Train the SVM
print("Build SVM")
xval_splitter = CDataSplitterKFold()
clf_lin = CClassifierSVM()


xval_lin_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

print("Find the best params")

best_lin_params = clf_lin.estimate_parameters(
    dataset = tr_set,
    parameters = xval_lin_params,
    splitter = xval_splitter,
    metric = 'accuracy',
    perf_evaluator = 'xval'
)


print("Finish Train")
print("The best training parameters are: ", [(k, best_lin_params[k]) for k in sorted(best_lin_params)])

print("Train SVM")
clf_lin.fit(tr_set.X, tr_set.Y)

Build SVM
Find the best params
Finish Train
The best training parameters are:  [('C', 1)]
Train SVM


CClassifierSVM{'classes': CArray(2,)(dense: [0 1]), 'n_features': 106158, 'preprocess': None, 'n_jobs': 1, 'C': 1.0, 'class_weight': None, 'w': CArray(1, 106158)(sparse: (0, 69804) 0.05520455534102185  (0, 25712) 0.11820530527221464  (0, 66307) 0.0008940752697913484  (0, 44977) 0.00017881505395826968  (0, 6022) 0.00017881505395826968  (0, 87439) 0.03884670023396364  (0, 60666) 0.0373479548374943  (0, 54394) 0.03884670023396364  (0, 44527) 0.03884670023396364  (0, 42072) 0.03884670023396364  (0, 34551) 0.03884670023396364  (0, 21489) 0.03289763676604037  (0, 95682) 0.032919656519926906  (0, 88629) 0.03176502575227214  (0, 62904) 0.0276828750877266  (0, 45555) 0.03368539104987918  (0, 90989) 0.1085386279274435  (0, 86726) 0.09191679949998977  (0, 46562) 0.1085386279274435  (0, 30632) 0.1085386279274435  (0, 813) 0.1085386279274435  (0, 88498) 0.07209930891627656  (0, 104490) 0.05411418692765332  (0, 104489) 0.057935291261985045  (0, 103734) 0.16234256078295994  : :  (0, 34708) 1.14803304

In [6]:
# Test the Classifier
ts_set = CDataset(tfidf_test, y_test)
print(ts_set)

y_pred = clf_lin.predict(ts_set.X)
metric = CMetricAccuracy()
acc = metric.performance_score(y_true=ts_set.Y, y_pred=y_pred)

confusion_matrix = CMetricConfusionMatrix() 
cm = confusion_matrix.performance_score(y_true=ts_set.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))
print("Confusion Matrix: ")
print(cm)
print("False Positive Rate: {:.2%}".format(39/(39+3445)))

print("False Negative Rate: {:.2%}".format(15/(15+3203)))

CDataset{'X': CArray(6744, 106158)(sparse: (0, 101946) 0.1060033440163194  (0, 101687) 0.4373061476555757  (0, 95734) 0.3208507119454746  (0, 95423) 0.31006036368134593  (0, 90296) 0.1891302217668824  (0, 88647) 0.22987985162514124  (0, 83685) 0.16079043665130593  (0, 82964) 0.2140125702818817  (0, 79609) 0.38704263644366776  (0, 76617) 0.16401947960473945  (0, 76396) 0.1630918680694897  (0, 68301) 0.3574896354751005  (0, 62550) 0.1105421599968617  (0, 62530) 0.1743943052625822  (0, 61791) 0.1690683682680177  (0, 19373) 0.189353740551165  (1, 96843) 0.12429398522034467  (1, 93422) 0.1292818095595453  (1, 92439) 0.108310870614031  (1, 92405) 0.1138709603876764  (1, 90820) 0.12429398522034467  (1, 89117) 0.11801006971601068  (1, 88909) 0.11801006971601068  (1, 88273) 0.06123721804043749  (1, 85516) 0.0733272612886512  : :  (6743, 87960) 0.11072450438215084  (6743, 82181) 0.08168787345627136  (6743, 78195) 0.08648793817854585  (6743, 77913) 0.203456033517438  (6743, 77758) 0.5120461034625

In [7]:
import random
from secml.array import CArray
from secml.adv.attacks.evasion import CAttackEvasionPGD

nb_attack=100

class_to_attack=1
cnt = 0 # the number of success adversaril examples

ori_examples2_x = []
ori_examples2_y = []
number_list = []
for i in range(nb_attack):
    #take a point at random being the starting point of the attack
    idx_candidates = np.where(y_test == class_to_attack)
    #select nb_init_pts points randomly in candidates and make them move
    rn = np.random.choice(idx_candidates[0].size, 1)
    x0,y0 =ts_set[idx_candidates[0][rn[0]],:].X, ts_set[idx_candidates[0][rn[0]],:].Y
    number_list.append(x_test_path[idx_candidates[0][rn[0]]])
    
    x0=x0.astype(float)
    y0=y0.astype(int)
    x2 = x0.tondarray()[0]
    y2 = y0.tondarray()[0]
    
    ori_examples2_x.append(x2)
    ori_examples2_y.append(y2)
    
number_list

[(10947,
  'spampy/datasets/enron\\enron2\\spam\\5512.2005-07-20.SA_and_HP.spam.txt'),
 (16415, 'spampy/datasets/enron\\enron3\\spam\\5096.2005-07-02.BG.spam.txt'),
 (29291, 'spampy/datasets/enron\\enron6\\spam\\0100.2004-08-07.BG.spam.txt'),
 (27528,
  'spampy/datasets/enron\\enron5\\spam\\4921.2005-07-20.SA_and_HP.spam.txt'),
 (29987, 'spampy/datasets/enron\\enron6\\spam\\1041.2004-10-14.BG.spam.txt'),
 (26511,
  'spampy/datasets/enron\\enron5\\spam\\3462.2005-07-04.SA_and_HP.spam.txt'),
 (10955,
  'spampy/datasets/enron\\enron2\\spam\\5542.2005-07-20.SA_and_HP.spam.txt'),
 (22053, 'spampy/datasets/enron\\enron4\\spam\\5348.2005-06-05.GP.spam.txt'),
 (19352, 'spampy/datasets/enron\\enron4\\spam\\1745.2004-07-14.GP.spam.txt'),
 (22501, 'spampy/datasets/enron\\enron4\\spam\\5947.2005-08-31.GP.spam.txt'),
 (11008,
  'spampy/datasets/enron\\enron2\\spam\\5765.2005-07-21.SA_and_HP.spam.txt'),
 (20936, 'spampy/datasets/enron\\enron4\\spam\\3860.2005-01-10.GP.spam.txt'),
 (27247,
  'spampy/

In [18]:
# Perform adversarial attacks
noise_type = 'l2'  # Type of perturbation 'l1' or 'l2'
dmax = 0.09 # Maximum perturbation
lb, ub = 0, 1  # Bounds of the attack space. Can be set to `None` for unbounded

solver_params = {
    'eta': 0.01,
    'max_iter': 20,
    'eps': 1e-6}

#set lower bound and upper bound respectively to 0 and 1 since all features are Boolean
pgd_attack = CAttackEvasionPGD(
    classifier=clf_lin,
    double_init_ds=tr_set,
    distance=noise_type,
    dmax=dmax,
    lb=lb, ub=ub,
    solver_params=solver_params)


ad_examples_x = []
ad_examples_y = []
cnt = 0
for i in range(len(ori_examples2_x)):
    print("Current Number:", i)
    x0 = ori_examples2_x[i]
    y0 = ori_examples2_y[i]

    y_pred_pgd, _, adv_ds_pgd, _ = pgd_attack.run(x0, y0)

    if y_pred_pgd.item() == 0:
        cnt = cnt + 1

    ad_examples_x.append(adv_ds_pgd.X.tondarray()[0])
    ad_examples_y.append(y_pred_pgd.item())

    attack_pt = adv_ds_pgd.X.tondarray()[0]
print("Accuracy:", cnt/nb_attack)

Current Number: 0
Current Number: 1
Current Number: 2
Current Number: 3
Current Number: 4
Current Number: 5
Current Number: 6
Current Number: 7
Current Number: 8
Current Number: 9
Current Number: 10
Current Number: 11
Current Number: 12
Current Number: 13
Current Number: 14
Current Number: 15
Current Number: 16
Current Number: 17
Current Number: 18
Current Number: 19
Current Number: 20
Current Number: 21
Current Number: 22
Current Number: 23
Current Number: 24
Current Number: 25
Current Number: 26
Current Number: 27
Current Number: 28
Current Number: 29
Current Number: 30
Current Number: 31
Current Number: 32
Current Number: 33
Current Number: 34
Current Number: 35
Current Number: 36
Current Number: 37
Current Number: 38
Current Number: 39
Current Number: 40
Current Number: 41
Current Number: 42
Current Number: 43
Current Number: 44
Current Number: 45
Current Number: 46
Current Number: 47
Current Number: 48
Current Number: 49
Current Number: 50
Current Number: 51
Current Number: 52
Cur

In [19]:
ori_examples2_x = np.array(ori_examples2_x)
ori_examples2_y = np.array(ori_examples2_y)
ad_examples_x = np.array(ad_examples_x)
ad_examples_y = np.array(ad_examples_y)

ori_dataframe = pd.DataFrame(ori_examples2_x, columns = vect.get_feature_names())
ad_dataframe = pd.DataFrame(ad_examples_x, columns = vect.get_feature_names())

# extract the success and fail examples
ad_dataframe['ad_label'] = ad_examples_y
ad_success = ad_dataframe.loc[ad_dataframe.ad_label == 0]
ori_success = ori_dataframe.loc[ad_dataframe.ad_label == 0]
ad_fail = ad_dataframe.loc[ad_dataframe.ad_label == 1]
ori_fail = ori_dataframe.loc[ad_dataframe.ad_label == 1]

ad_success_x = ad_success.drop(columns = ['ad_label'])
ad_fail_x = ad_fail.drop(columns = ['ad_label'])
result = (ad_success_x - ori_success)

vect.idf_
IDF = pd.DataFrame(vect.idf_.T, index=vect.get_feature_names())
IDF.to_csv("idf.csv")
IDF

,0
aa,5.987656
aaa,7.676231
aaaa,10.103979
aaaahhhhhh,10.509444
aaadrizzl,10.103979
...,...
zzxtfeerekvwkug,10.509444
zzxxst,10.509444
zzzglvaa,10.509444
zzzz,8.258153


In [11]:
# Method 1
import math
success_num = 0 # the number of success of our method 1
current_num = 0
left = 0
ad_all_x = pd.DataFrame(ad_success_x, columns = vect.get_feature_names())
m1_empty = pd.DataFrame() # the formulated tf-idf matrix via method 1
for left in range(cnt):
    print(current_num)
    right = left+1
    extract100 = ad_success_x[left:right].values/vect.idf_    # tf-idf[i] / idf[i]
    extract100_ad = ad_success_x[left:right]
    extract100_ad = extract100_ad.loc[:, (extract100_ad>0).all(axis=0)] # Extract the features of TF-IDF > 0
    extract100_ad_columns = extract100_ad.columns.tolist() # the list of above features     
    extract100_ad_idf = IDF.loc[extract100_ad_columns] # the idf value of above features

    X_idf5 = extract100_ad_idf.index.tolist()


    print("len_column: ", len(extract100_ad_columns))
    extract100 = pd.DataFrame(extract100, columns = vect.get_feature_names())
    total_tf = extract100.sum().sum()
    print("total_tf", extract100.sum().sum())
    
    
    for index, row in ad_success_x[left:right].iterrows():
        row = row.sort_values(axis=0, ascending = False)
    l = row[0:500].index.tolist()     # the 500 words with the largest tf-idf value

    X_idf5 = list(set(X_idf5).difference(set(l)))  # delete the words in top 500 words

    extract100 = extract100[l]
    extract100.to_csv("extract100.csv")


    extract100_list = extract100.values.tolist()[0]
    c = len(X_idf5)
    c2 =  float(extract100.quantile(axis=1))    # c2 = a/IDF[A]
    ratio = [] # the ratio between word A and other words (Get b1, c1, d1, ...)
    for i in range(500):
        ratio.append(extract100_list[i]/c2)
    extract100_sum = sum(ratio)     # = 1+b1+c1+d1+......
    print("Extract100_sum", extract100_sum)
    print("c2", c2)
    y1 = (total_tf/c2-extract100_sum)/(c)
    print("y1", y1)
    
    
    flag = True
    for y in range(1,1000):
        for A in range(1,5000):
            if abs(y/A - y1) < 1e-5:
                print(y, y1*A)
                flag = False
                break
        if flag == False:
            break

    print(flag, A, y, y/A)

    extract100_re = []
    for j in range(500):
        extract100_re.append(ratio[j]*A)

    extract100_re_list = extract100_re
    extract100_re = pd.DataFrame(extract100_re, index = l)
    extract100_re.to_csv("extract100_re.csv")
    
    
    
    
    lint = []
    total = 0
    for item in extract100_re_list:
        newitem = float(item)
        lint.append(round(newitem))
        total = total + round(newitem)

    len_y = [math.ceil(y)]*c
    lint2 = lint+len_y
    # print(lint2)
    total2 = total + c*y
    lint2 = [i/total2 for i in lint2]
    l2 = l+X_idf5

    dictionary = dict(zip(l2, lint2))
    formulate_idf = IDF.loc[dictionary.keys()][0].tolist()
    formulate_tf = list(dictionary.values())
    tf_idf_np = np.multiply(np.array(formulate_idf), np.array(formulate_tf))
    tf_idf_np.tolist()

    formulate_tf_idf = pd.DataFrame(tf_idf_np.tolist(), index = list(dictionary.keys()))
    formulate_tf_idf = pd.DataFrame(formulate_tf_idf.values.T, index = formulate_tf_idf.columns, columns = formulate_tf_idf.index)
    formulate_tf_idf = formulate_tf_idf.loc[0:1]*total_tf 
    formulate_columns = formulate_tf_idf.columns
    formulate_tf_idf


    ad_df = ad_success_x[left:right][formulate_columns.tolist()]
    other_df = ad_success_x[left:right].drop(formulate_columns.tolist(), axis=1)
    other_columns = other_df.columns.tolist()
    all_columns_order = formulate_columns.tolist()+other_columns
    ad_df = pd.concat([ad_df, other_df], axis = 1)


    formulate0 = [0]* len(other_columns)
    formulate0_pf = pd.DataFrame(formulate0, index = other_columns)
    formulate0_pf = pd.DataFrame(formulate0_pf.values.T, columns = formulate0_pf.index, index = formulate0_pf.columns)

    df = pd.concat([formulate_tf_idf, formulate0_pf], axis = 1)
    df.to_csv("formulate.csv")

    df_empty = pd.DataFrame(columns=vect.get_feature_names(), dtype=np.float)
    df_empty = df_empty.append(df, sort=True)
    
    formulate_y = [1]
    formulate_y = pd.Series(formulate_y)
    formulate_set = CDataset(df_empty, formulate_y)
    
    m1_empty = m1_empty.append(df_empty, ignore_index=True)

    # Predict
    formulate_pred = clf_lin.predict(formulate_set.X)
    if formulate_pred == 0:
        success_num = success_num+1
    print(formulate_pred)
    current_num = current_num+1
print(success_num)
m1_empty

0
len_column:  11066
total_tf 1.908844010867873
Extract100_sum 3385.3303878243837
c2 0.0004463996503093037
y1 0.08430410899264693
29 29.000613493470542
False 344 29 0.08430232558139535
CArray([0])
1
len_column:  11093
total_tf 1.9216606386524315
Extract100_sum 3188.41063826643
c2 0.0004758328231000712
y1 0.08025204574309293
13 13.000831410381055
False 162 13 0.08024691358024691
CArray([0])
2
len_column:  11103
total_tf 2.0980077200827854
Extract100_sum 3363.592657150503
c2 0.0005025663763224427
y1 0.07648737657461312
27 27.00004393083843
False 353 27 0.0764872521246459
CArray([0])
3
len_column:  11075
total_tf 2.249389288557292
Extract100_sum 3995.8001540530113
c2 0.00046300866140121146
y1 0.08155093738542984
19 19.001368410805153
False 233 19 0.0815450643776824
CArray([0])
4
len_column:  11067
total_tf 1.6494672319262154
Extract100_sum 2876.9724269106355
c2 0.00043554418134784045
y1 0.08613311115377839
18 18.001820231139682
False 209 18 0.0861244019138756
CArray([0])
5
len_column:  11

41 41.003839796714765
False 443 41 0.09255079006772009
CArray([0])
42
len_column:  11122
total_tf 2.916022999624117
Extract100_sum 4167.065403544957
c2 0.0006004190598547872
y1 0.06492006088119133
37 37.004434702279056
False 570 37 0.06491228070175438
CArray([0])
43
len_column:  11054
total_tf 1.4702523793323616
Extract100_sum 2600.1100573902263
c2 0.0004145339852394366
y1 0.08969580375562808
20 20.002164237505063
False 223 20 0.08968609865470852
CArray([0])
44
len_column:  11059
total_tf 1.8659254150739915
Extract100_sum 3467.6628249146365
c2 0.0004242913370156458
y1 0.08808437387626591
17 17.00028415811932
False 193 17 0.08808290155440414
CArray([0])
45
len_column:  11041
total_tf 1.3096724784959977
Extract100_sum 2290.7219603559324
c2 0.00040149892500869954
y1 0.09213885411661409
34 33.9992371690306
False 369 34 0.0921409214092141
CArray([0])
46
len_column:  11123
total_tf 2.5146048951987208
Extract100_sum 3404.600819589512
c2 0.000616004871800207
y1 0.06377837708576065
25 25.001123

CArray([0])
84
len_column:  11053
total_tf 1.573042718622193
Extract100_sum 2848.8834287590194
c2 0.00041445962688651324
y1 0.08969231184375431
20 20.001385541157212
False 223 20 0.08968609865470852
CArray([1])
85
len_column:  11058
total_tf 1.882039140486258
Extract100_sum 3496.6933244203797
c2 0.0004256520737566546
y1 0.08759709507308724
12 12.000802025012952
False 137 12 0.08759124087591241
CArray([0])
86
len_column:  11061
total_tf 2.393428413445259
Extract100_sum 4522.261575531958
c2 0.0004420897382931317
y1 0.08442720038482177
32 31.99790894584745
False 379 32 0.08443271767810026
CArray([0])
87
len_column:  11053
total_tf 1.6084643174128812
Extract100_sum 2921.331883647706
c2 0.00041618031450928617
y1 0.0894052570019656
27 27.00038761459361
False 302 27 0.08940397350993377
CArray([0])
88
len_column:  11067
total_tf 1.5879011575027755
Extract100_sum 2724.336800012342
c2 0.0004373913810597959
y1 0.08574362483253005
65 64.99366762305777
False 758 65 0.08575197889182058
CArray([0])
8

,aa,aaa,aaaa,aaadrizzl,aaagrp,aaaplusdirect,aaasash,aab,aababp,aabda,...,zzucpkow,zzw,zzx,zzxtfeerekvwkug,zzxxst,zzyudgpd,zzzglvaa,zzzxlqbha,zzzz,zzzzcard
0,0.0,0.000295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.0,0.000311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,0.0,0.000293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.000291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,0.0,0.000298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Black-box Attack

from collections import namedtuple
CLF = namedtuple('CLF', 'clf_name clf xval_parameters')

random_state = 999
from secml.ml.classifiers import CClassifierKNN, CClassifierDecisionTree, CClassifierSGD


target_clf_list = [
    CLF(clf_name='kNN',
        clf=CClassifierKNN(),
        xval_parameters={'n_neighbors': [160]}),
    CLF(clf_name='Decision Tree',
        clf=CClassifierDecisionTree(random_state=random_state),
        xval_parameters={'max_depth': [55]}),
    CLF(clf_name='Logistic(SGD)',
        clf=CClassifierSGD(random_state=random_state, regularizer = 'l2', loss='log'),
        xval_parameters={'alpha': [1e-6, 1e-5, 1e-4, 1e-4]}),
]

for i, test_case in enumerate(target_clf_list):

    clf = test_case.clf
    xval_params = test_case.xval_parameters

    print("\nEstimating the best training parameters of {:} ..."
          "".format(test_case.clf_name))

    best_params2 = clf.estimate_parameters(
        dataset=tr_set, parameters=xval_params, splitter=xval_splitter,
        metric='accuracy', perf_evaluator='xval')

    print("The best parameters for '{:}' are: ".format(test_case.clf_name),
          [(k, best_params2[k]) for k in sorted(best_params2)])

    print("Training of {:} ...".format(test_case.clf_name))
    clf.fit(tr_set.X, tr_set.Y)

    # Predictions on test set and performance evaluation
    y_pred1 = clf.predict(ts_set.X)
    acc1 = metric.performance_score(y_true=ts_set.Y, y_pred=y_pred1)

    print("Classifier: {:}\tAccuracy: {:.2%}".format(test_case.clf_name, acc1))

In [ ]:
ori_dataframe
ori_100 = CDataset(ori_dataframe, ori_examples2_y)


PGD_y = [0]*len(ad_success_x)
PGD_y = pd.Series(PGD_y)
PGD_100 = CDataset(ad_success_x, PGD_y)

m1_y = [0]*len(m1_empty)
m1_y = pd.Series(m1_y)
m1_91 = CDataset(m1_empty, m1_y)


for target_clf in target_clf_list:

    # original emails
    y100 = target_clf.clf.predict(ori_100.X)
    acc100 = metric.performance_score(y_true=ori_100.Y, y_pred=y100)
    print("Classifier: {:}\tAccuracy of 100: {:.2%}".format(target_clf.clf_name, acc100))
    
    #PGD
    yPGD = target_clf.clf.predict(PGD_100.X)
    accPGD = metric.performance_score(y_true=PGD_100.Y, y_pred=yPGD)
    print("Classifier: {:}\tAccuracy of PGD: {:.2%}".format(target_clf.clf_name, accPGD))
    
    y_m1 = target_clf.clf.predict(m1_91.X)
    acc_m1 = metric.performance_score(y_true=m1_91.Y, y_pred=y_m1)
    print("Classifier: {:}\tAccuracy of Method 1: {:.2%}".format(target_clf.clf_name, acc_m1))
    